<a href="https://colab.research.google.com/github/honoreade/DeepLearning/blob/main/tf_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computing Transfer leaning**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Dependency libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create ImageDataGenerator instances for training, validation, and test sets
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=30.0,
    fill_mode="nearest"
)

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)  # Only rescale for validation and test

# Define paths
train_data_path = "/content/drive/MyDrive/Colab Notebooks/FruitClassifier/FruitClassifier/train"
val_data_path = "/content/drive/MyDrive/Colab Notebooks/FruitClassifier/FruitClassifier/val"
test_data_path = "/content/drive/MyDrive/Colab Notebooks/FruitClassifier/FruitClassifier/test"

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical"  # For multi-class classification
)

val_generator = val_test_datagen.flow_from_directory(
    val_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = val_test_datagen.flow_from_directory(
    test_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False  # No need to shuffle test data
)

Found 2489 images belonging to 7 classes.
Found 720 images belonging to 7 classes.
Found 1400 images belonging to 7 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Load pre-trained ResNet50 with a defined input shape
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze all convolutional base layers initially
for layer in base_model.layers:
    layer.trainable = False

# Add new fully connected layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu", kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
output = Dense(train_generator.num_classes, activation="softmax")(x)  # Dynamically set class count

# Create the new model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=3e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping # Import EarlyStopping

# Define Callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train only the new fully connected layers first
model.fit(train_generator,
          epochs=10,
          validation_data=val_generator,
          callbacks=[early_stopping])

Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.2854 - loss: 2.6532 - val_accuracy: 0.2347 - val_loss: 2.4729 - learning_rate: 3.0000e-04
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 70s 710ms/step - accuracy: 0.3083 - loss: 2.2607 - val_accuracy: 0.2667 - val_loss: 2.2845 - learning_rate: 3.0000e-04
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 712ms/step - accuracy: 0.3729 - loss: 2.0801 - val_accuracy: 0.1458 - val_loss: 2.6671 - learning_rate: 3.0000e-04
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 707ms/step - accuracy: 0.3603 - loss: 2.0450 - val_accuracy: 0.1833 - val_loss: 2.4038 - learning_rate: 3.0000e-04
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 704ms/step - accuracy: 0.3764 - loss: 1.9849 - val_accuracy: 0.2194 - val_loss: 2.6553 - learning_rate: 3.0000e-04
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 721ms/step - accuracy: 0.3718 - loss: 1.9370 - val_accuracy: 0.3139 - val_loss: 2.0304 - learning_rate: 6.0000e-05
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 718ms/step - accur

### Fine-Tune with the 40% of pretrained layers

In [ ]:
# Unfreeze some deeper layers for fine-tuning
for layer in base_model.layers[-20:]:  # Unfreeze last 20 layers
    layer.trainable = True

# Compile with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Continue training with fine-tuning
model.fit(train_generator,
          epochs=10,
          validation_data=val_generator,
          callbacks=[early_stopping])

Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 93s 935ms/step - accuracy: 0.2739 - loss: 2.2919 - val_accuracy: 0.1431 - val_loss: 7.8872
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 58s 746ms/step - accuracy: 0.3527 - loss: 2.0136 - val_accuracy: 0.2250 - val_loss: 4.9294
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 725ms/step - accuracy: 0.3640 - loss: 1.9111 - val_accuracy: 0.1819 - val_loss: 4.3367
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 57s 727ms/step - accuracy: 0.3893 - loss: 1.8730 - val_accuracy: 0.2931 - val_loss: 2.7461
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 717ms/step - accuracy: 0.4098 - loss: 1.8468 - val_accuracy: 0.3542 - val_loss: 2.0872
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 81s 712ms/step - accuracy: 0.4252 - loss: 1.7858 - val_accuracy: 0.4222 - val_loss: 1.8026
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 704ms/step - accuracy: 0.4393 - loss: 1.7826 - val_accuracy: 0.4556 - val_loss: 1.7096
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 700ms/step - accuracy: 0.4509 - loss: 1.7401 - val_accu

### **Try Out Bunch of other Pretained Models**

In [ ]:
from tensorflow.keras.applications import ResNet50, EfficientNetB0, EfficientNetB1, EfficientNetB6, EfficientNetB7, DenseNet121, DenseNet169, DenseNet201, InceptionV3, ResNet101, ResNet152, VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

def train_pretrained_model(model_name, train_generator, validation_generator, epochs=10):
    # Dictionary to store model architectures
    model_dict = {
        "VGG19": VGG19,
        "ResNet50": ResNet50,
        "ResNet101": ResNet101,
        "ResNet152": ResNet152,
        "EfficientNetB0": EfficientNetB0,
        "EfficientNetB1": EfficientNetB1,
        "EfficientNetB6": EfficientNetB6,
        "EfficientNetB7": EfficientNetB7,
        "DenseNet121": DenseNet121,
        "DenseNet169": DenseNet169,
        "DenseNet201": DenseNet201,
        "InceptionV3": InceptionV3
    }

    if model_name not in model_dict:
        raise ValueError(f"Invalid model name: {model_name}. Choose from {list(model_dict.keys())}")

    base_model = model_dict[model_name](weights="imagenet", include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Fully connected layers with L2 regularization and batch normalization
    x = Dense(128, activation="relu", kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)  # Dropout to reduce overfitting
    output = Dense(train_generator.num_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

    print(f"Starting training for {model_name}...")
    history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=[early_stopping])

    print(f"Training complete for {model_name}. Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")
    print(f"Final validation loss: {history.history['val_loss'][-1]:.4f}")

    return model

def save_model(model, model_name, save_dir="saved_models"):
    """Saves the trained model to the specified directory."""
    import os
    os.makedirs(save_dir, exist_ok=True)
    model_path = os.path.join(save_dir, f"{model_name}.h5")
    model.save(model_path)
    print(f"Model {model_name} saved at {model_path}")


# Example usage (assuming you have train_generator and validation_generator defined):
models_to_train = [
    "VGG19", "ResNet50", "ResNet101", "ResNet152",
    "EfficientNetB0", "EfficientNetB1", "EfficientNetB6", "EfficientNetB7",
    "DenseNet121", "DenseNet169", "DenseNet201", "InceptionV3"
]

for model_name in models_to_train:
    trained_model = train_pretrained_model(model_name, train_generator, val_generator)
    save_model(trained_model, model_name)


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Starting training for VGG19...
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.3877 - loss: 1.9378 - val_accuracy: 0.3333 - val_loss: 1.6765
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 58s 740ms/step - accuracy: 0.6690 - loss: 1.1422 - val_accuracy: 0.6819 - val_loss: 1.3368
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 82s 733ms/step - accuracy: 0.7081 - loss: 1.0038 - val_accuracy: 0.6875 - val_loss: 1.1173
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 58s 738ms/step - accuracy: 0.7227 - loss: 0.9480 - val_accuracy: 0.7194 - val_loss: 0.9643
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 58s 736ms/step - accuracy: 0.7417 - loss: 0.8860 - val_accuracy: 0.5153 - val_loss: 1.5633
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 63s 810ms/step - accuracy: 0.7459 - loss: 0.8698 - val_accuracy: 0.7764 - val_loss: 0.7980
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 57s 736ms/step - accuracy: 0.7577 - loss: 0.8669 - val_accuracy: 0.7778 - val_loss: 0.8091
Epoch 8/10
78/78 ━

Training complete for VGG19. Final validation accuracy: 0.8097
Final validation loss: 0.7292
Model VGG19 saved at saved_models/VGG19.h5
Starting training for ResNet50...
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 78s 843ms/step - accuracy: 0.2061 - loss: 2.1711 - val_accuracy: 0.1472 - val_loss: 2.4162
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 717ms/step - accuracy: 0.3325 - loss: 1.8653 - val_accuracy: 0.1708 - val_loss: 2.3621
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 57s 727ms/step - accuracy: 0.3423 - loss: 1.8260 - val_accuracy: 0.2625 - val_loss: 2.1837
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 716ms/step - accuracy: 0.3542 - loss: 1.7433 - val_accuracy: 0.2722 - val_loss: 2.2982
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 709ms/step - accuracy: 0.3793 - loss: 1.6939 - val_accuracy: 0.2736 - val_loss: 1.9077
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 723ms/step - accuracy: 0.4369 - loss: 1.6199 - val_accuracy: 0.3250 - val_loss: 1.9020
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 61s 779ms/step - a

Training complete for ResNet50. Final validation accuracy: 0.2069
Final validation loss: 3.6620
Model ResNet50 saved at saved_models/ResNet50.h5
171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
Starting training for ResNet101...
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 96s 957ms/step - accuracy: 0.2047 - loss: 2.2035 - val_accuracy: 0.1806 - val_loss: 2.0858
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 58s 732ms/step - accuracy: 0.2751 - loss: 1.9370 - val_accuracy: 0.2278 - val_loss: 1.9238
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 82s 740ms/step - accuracy: 0.3158 - loss: 1.8534 - val_accuracy: 0.1750 - val_loss: 2.0423
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 57s 732ms/step - accuracy: 0.2993 - loss: 1.8402 - val_accuracy: 0.2417 - val_loss: 2.0160
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 57s 733ms/step - accuracy: 0.3334 - loss: 1.8129 - val_accuracy: 0.2181 - val_loss: 2.3525
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 59s 752ms/step - accuracy: 0.3250 - loss: 1.7745 - val_accuracy: 0.3444 - val_loss: 

Training complete for ResNet101. Final validation accuracy: 0.2167
Final validation loss: 2.1011
Model ResNet101 saved at saved_models/ResNet101.h5
234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
Starting training for ResNet152...
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.1830 - loss: 2.2897 - val_accuracy: 0.1667 - val_loss: 2.1065
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 60s 768ms/step - accuracy: 0.2473 - loss: 1.9919 - val_accuracy: 0.2014 - val_loss: 2.1189
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 61s 775ms/step - accuracy: 0.3247 - loss: 1.8267 - val_accuracy: 0.2097 - val_loss: 1.9682
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 60s 765ms/step - accuracy: 0.2989 - loss: 1.8483 - val_accuracy: 0.2069 - val_loss: 2.2492
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 65s 830ms/step - accuracy: 0.3264 - loss: 1.8203 - val_accuracy: 0.2500 - val_loss: 1.9238
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 59s 760ms/step - accuracy: 0.3322 - loss: 1.8184 - val_accuracy: 0.1694 - val_loss

Training complete for ResNet152. Final validation accuracy: 0.2236
Final validation loss: 2.5864
Model ResNet152 saved at saved_models/ResNet152.h5
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Starting training for EfficientNetB0...
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 102s 996ms/step - accuracy: 0.1401 - loss: 2.3572 - val_accuracy: 0.1389 - val_loss: 2.1672
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 704ms/step - accuracy: 0.1396 - loss: 2.2168 - val_accuracy: 0.1667 - val_loss: 2.1480
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 700ms/step - accuracy: 0.1484 - loss: 2.1945 - val_accuracy: 0.1389 - val_loss: 2.1968
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 709ms/step - accuracy: 0.1531 - loss: 2.1701 - val_accuracy: 0.1389 - val_loss: 2.1526
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 713ms/step - accuracy: 0.1640 - loss: 2.1387 - val_accuracy: 0.1389 - val_loss: 2.3715
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 713ms/step - accuracy: 0.1613 - loss: 2.1341 - val_accuracy: 0.1444 - val

...

Training complete for EfficientNetB7. Final validation accuracy: 0.1458
Final validation loss: 2.0024
Model EfficientNetB7 saved at saved_models/EfficientNetB7.h5
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Starting training for DenseNet121...
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.5645 - loss: 1.5457 - val_accuracy: 0.8625 - val_loss: 0.5312
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 710ms/step - accuracy: 0.8693 - loss: 0.5288 - val_accuracy: 0.9236 - val_loss: 0.3657
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 82s 707ms/step - accuracy: 0.8922 - loss: 0.4593 - val_accuracy: 0.9278 - val_loss: 0.3528
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 708ms/step - accuracy: 0.9199 - loss: 0.3806 - val_accuracy: 0.9389 - val_loss: 0.3229
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 714ms/step - accuracy: 0.8988 - loss: 0.3928 - val_accuracy: 0.9361 - val_loss: 0.3101
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 707ms/step - accuracy: 0.9175 - loss: 0.3529 - val_accuracy: 0.9

Training complete for DenseNet121. Final validation accuracy: 0.9111
Final validation loss: 0.3870
Model DenseNet121 saved at saved_models/DenseNet121.h5
51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Starting training for DenseNet169...
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.6181 - loss: 1.3324 - val_accuracy: 0.8986 - val_loss: 0.4449
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 713ms/step - accuracy: 0.8817 - loss: 0.4506 - val_accuracy: 0.9292 - val_loss: 0.3664
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 57s 730ms/step - accuracy: 0.9092 - loss: 0.3816 - val_accuracy: 0.9208 - val_loss: 0.3636
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 721ms/step - accuracy: 0.9399 - loss: 0.3080 - val_accuracy: 0.9236 - val_loss: 0.3389
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 56s 723ms/step - accuracy: 0.9422 - loss: 0.2987 - val_accuracy: 0.9444 - val_loss: 0.3095
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 57s 729ms/step - accuracy: 0.9394 - loss: 0.3013 - val_accuracy: 0.9431 - val

Training complete for DenseNet169. Final validation accuracy: 0.9319
Final validation loss: 0.3335
Model DenseNet169 saved at saved_models/DenseNet169.h5
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Starting training for DenseNet201...
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.6511 - loss: 1.2658 - val_accuracy: 0.8542 - val_loss: 0.5607
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 58s 739ms/step - accuracy: 0.8829 - loss: 0.4718 - val_accuracy: 0.9042 - val_loss: 0.4143
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 59s 751ms/step - accuracy: 0.9154 - loss: 0.3685 - val_accuracy: 0.9236 - val_loss: 0.3375
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 57s 732ms/step - accuracy: 0.9336 - loss: 0.3057 - val_accuracy: 0.9306 - val_loss: 0.3280
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 82s 730ms/step - accuracy: 0.9376 - loss: 0.2930 - val_accuracy: 0.9403 - val_loss: 0.2873
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 58s 745ms/step - accuracy: 0.9526 - loss: 0.2570 - val_accuracy: 0.9167 - val

Training complete for DenseNet201. Final validation accuracy: 0.9333
Final validation loss: 0.3237
Model DenseNet201 saved at saved_models/DenseNet201.h5
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Starting training for InceptionV3...
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.6330 - loss: 1.4055 - val_accuracy: 0.8583 - val_loss: 0.6405
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 59s 758ms/step - accuracy: 0.8285 - loss: 0.7817 - val_accuracy: 0.9042 - val_loss: 0.5004
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 82s 754ms/step - accuracy: 0.8652 - loss: 0.6627 - val_accuracy: 0.9097 - val_loss: 0.4820
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 59s 752ms/step - accuracy: 0.8664 - loss: 0.6339 - val_accuracy: 0.9167 - val_loss: 0.4979
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 60s 764ms/step - accuracy: 0.8814 - loss: 0.5995 - val_accuracy: 0.8889 - val_loss: 0.5538
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 60s 772ms/step - accuracy: 0.8694 - loss: 0.5893 - val_accuracy: 0.9083 - val

Training complete for InceptionV3. Final validation accuracy: 0.8639
Final validation loss: 0.5722
Model InceptionV3 saved at saved_models/InceptionV3.h5


In [ ]:
import shutil

# Define source and destination folders
source_folder = "/content/saved_models"
destination_folder = "/content/drive/MyDrive/Colab Notebooks/saved_models"

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Copy all files from source to destination
for file_name in os.listdir(source_folder):
    shutil.copy(os.path.join(source_folder, file_name), os.path.join(destination_folder, file_name))

print("Models copied to Google Drive successfully!")


Models copied to Google Drive successfully!


### **Making inference on top 5 performing Models based on validation Accuracy**

In [ ]:
import os

def load_models(directory, model_names):
    """Loads only .h5 models from the specified directory that match given model names exactly."""
    return {
        os.path.splitext(f)[0]: tf.keras.models.load_model(os.path.join(directory, f))
        for f in os.listdir(directory)
        if f.endswith(".h5") and os.path.splitext(f)[0] in model_names
    }

def evaluate_models(models, test_generator):
    """Evaluates models on the test dataset and returns results."""
    return {
        name: dict(zip(["Loss", "Accuracy"], model.evaluate(test_generator)))
        for name, model in models.items()
    }

def print_results(results):
    """Prints the evaluation results."""
    print("\n=== Test Set Evaluation Results ===")
    for name, metrics in results.items():
        print(f"{name}: Loss = {metrics['Loss']:.4f}, Accuracy = {metrics['Accuracy']:.2%}")

# Define model directory and exact model names to load
saved_models_dir = "/content/drive/MyDrive/Colab Notebooks/saved_models"
target_models = ["VGG19", "DenseNet121", "DenseNet169", "DenseNet201", "InceptionV3"]

# Load, evaluate, and print results for selected models
models = load_models(saved_models_dir, target_models)
results = evaluate_models(models, test_generator)
print_results(results)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


44/44 ━━━━━━━━━━━━━━━━━━━━ 372s 8s/step - accuracy: 0.9098 - loss: 0.4239
44/44 ━━━━━━━━━━━━━━━━━━━━ 1072s 24s/step - accuracy: 0.7978 - loss: 0.7221
44/44 ━━━━━━━━━━━━━━━━━━━━ 314s 7s/step - accuracy: 0.9224 - loss: 0.4046
44/44 ━━━━━━━━━━━━━━━━━━━━ 182s 4s/step - accuracy: 0.8903 - loss: 0.5292
44/44 ━━━━━━━━━━━━━━━━━━━━ 389s 9s/step - accuracy: 0.9266 - loss: 0.3358

=== Test Set Evaluation Results ===
DenseNet121: Loss = 0.3801, Accuracy = 92.21%
VGG19: Loss = 0.7600, Accuracy = 79.71%
DenseNet169: Loss = 0.3794, Accuracy = 92.64%
InceptionV3: Loss = 0.5219, Accuracy = 89.50%
DenseNet201: Loss = 0.3234, Accuracy = 93.36%


### **These models seem more promising and if we finetune with half the Pretrained Models unfrozen, that would almost reach to 100%.**

### **Shout to Regularization Techniques that were applied namely L2 regularization, batch normalization, and Dropout to reduce overfitting!**

##### **End.**